In [1]:
# Libraries used in the model are imported here for easy access
import pandas as pd


# Contents

# Introduction
 In this project, I take the CRISP-DM (Cross-Industry Standard Process for Data Mining) approach to extract value from business data. The case study is an online retail company registered in the UK with customers from various countries. The raw dataset used for the project is available at the [UCI data repository](https://archive.ics.uci.edu/dataset/352/online+retail).  

The focus of this project is to demonstrate expertise in Data Management and Machine Learning operations. The project includes the establishment of an analytical Dimensional Database model using the star schema. To create more fields for the Database model, the base  dataset from UCI is augmented with generated data using the python faker library. Ultimately, only the original data is queried from the denormalized database and used to create a segmentation and recommendation system for the business.

# CRISP-DM: Business Understanding

Like any typical online business, cross-selling and up-selling to existing customers is a key success factor. From their data records, the company in this case study has thousands of customers. It is inpractical to review their preferences manually and make related recommendations. It is also important for an online retail company to be aware of the habits of their customers so that they can tailor their advertisements and promotions to specific and relevant customer groups. An automated system that understands the customer's behaviour based on previous purchasing decisions will help the company resolve this challenge. The system could also go further to recommend relevant products to the customers. It is best if this recommendation takes place while the customer is still on the website, rather than via email campaigns or subsequent phone calls. The system therefore has to function in real-time.

# CRISP-DM: Data Understanding
